In [16]:
class Tree:
    """A tree with label as its label value."""
    def __init__(self, label, branches=[]):
        self.label = label
        for branch in branches:
            assert isinstance(branch, Tree)
        self.branches = list(branches)

    def __repr__(self):
        if self.branches:
            branch_str = ', ' + repr(self.branches)
        else:
            branch_str = ''
        return 'Tree({0}{1})'.format(self.label, branch_str)

    def __str__(self):
        return '\n'.join(self.indented())

    def indented(self, k=0):
        indented = []
        for b in self.branches:
            for line in b.indented(k + 1):
                indented.append('  ' + line)
        return [str(self.label)] + indented

    def is_leaf(self):
        return not self.branches

In [17]:
class BTree(Tree): #Subclass of a Tree
    empty = Tree(None) # Represent an empty as a Tree that contains nothing
    
    def __init__(self, label, left = empty, right = empty):
        # Construct a binary tree
        Tree.__init__(self, label, [left, right])
        
    # To access left branch, we use @property method decorator
    @property
    def left(self):
        return self.branches[0]
    
    # Same for right branch
    @property
    def right(self):
        return self.branches[1]
    
    # A new definition of is_leaf
    def is_leaf(self):
        # A leaf is when both branches are empty
        return [self.left, self.right] == [BTree.empty] * 2
    
    # The __repr__ method shows what to display
    def __repr__(self):
        if self.is_leaf():
            return 'BTree({0})'.format(self.label)
        elif self.right is BTree.empty:
            left = repr(self.left)
            return 'BTree({0}, {1})'.format(self.label, left)
        else:
            left, right = repr(self.left), repr(self.right)
            if self.left is BTree.empty:
                left = 'BTree.empty' 
            template = 'BTree({0}, {1}, {2})'
            return template.format(self.label, left, right)

In [18]:
def balanced_bst(s):
    # Base case: if s is empty, return an empty binary search tree
    if not s:
        return BTree.empty
    else:
        #First, find the midpoint
        mid = len(s) // 2
        # Then we construct a binary search tree out of everything that's less than the midpoint

        # For left branch, create a bst that's up to but not including the midpoint
        left = balanced_bst(s[:mid])
        # For the right tree includes the elements after midpoint, and all the elements after that.
        right = balanced_bst(s[mid+1:])
        # Return a BTree with s[mid] as the label
        return BTree(s[mid], left, right)

# Sets as Binary Search Trees

## Membership in Binary Search Trees

`contains` traverses the tree 
* If the element is not the root, it can only be in either the left or right branch
* By focusing on one branch, we reduce the set by the size of the other branch

So let's say we have the following tree,

<img src = 'bin_tree.jpg' width = 200/>

And we want to know if `9` is in the set. How do we implement `contains`?

In [19]:
def contains(s, v):
    # If s is empty, than v is definitely not there
    if s is BTree.empty:
        return False
    # If the label of the currently selected Tree is v, then we found it!
    elif s.label == v:
        return True
    # If v is smaller than the label of the currently selected Tree, search through the left branch of the current Tree
    elif s.label > v:
        return contains(s.left, v)
    # if v is greater than the label of the currently selected Tree, search through the right branch of the current Tree
    elif s.label < v:
        return contains(s.right, v)

What's the order of growth? Depends on the structure of the tree. 
1. On average, it's $\Theta(h)$ where $h$ is the height of the tree
    * We traverse from branch to branch, picking one branch at a time
2. A balanced tree (roughly same elements on the left and right branches) on average has a height of $log(n)$, where `n` is the number of entries the tree contains.
    * Thus, a balanced tree has an order of growth of $\Theta(log(n))$, which is faster than the linear time that we saw for an ordered list.

## Adjoining to a Tree Set

Why wouldn't we use a sorted list in the first place then perform binary search on that?

If we did that, `adjoin` would become slow. Adding an element into the middle of a list takes a long time.
* It's fast on a linked list, but we can't perform binary search on a linked list
* If it's a regular list, it would be slow

However, adjoining to a binary search tree is fast!

Let's say we want to adjoin `8` to the following tree,

<img src = 'adjoin_1.jpg' width = 150/>

Since `8` is greater than `5`, we go to the right branch.

<img src = 'adjoin_2.jpg' width = 100/>

Above, since 8 is less than 9, we take the left branch. Now we have 7 left. Technically, this 7 still has branches: empty branches.

<img src = 'adjoin_3.jpg' width = 200/>

Since 8 is greater than 7, we take the right branch!

<img src = 'adjoin_4.jpg' width = 100/>

Now since we're left with an empty Tree, we put 8 there. Thus, our new binary search tree would be the following,

<img src = 'adjoin_5.jpg' width = 400/>

## Implementation

We might think the following implementation would work. However, this implementation is wrong!

In [20]:
def adjoin(s, v): # s is a set represented as binary search tree
    # if s is an empty BTree
    if s is BTree.empty:
        return BTree(v) # Then add v to this tree
    # If v is already in the tree, then do nothing
    elif s.label == v:
        return s
    # If the current tree label is smaller than v, create a binary 
    elif s.label < v:
        return adjoin(s.right, v)
    # Similar for the left side
    elif s.label > v:
        return adjoin(s.left, v)

How could it be wrong?

In [21]:
odds = [2 * n + 1 for n in range(6)]
odds

[1, 3, 5, 7, 9, 11]

In [22]:
t = balanced_bst(odds)
t

BTree(7, BTree(3, BTree(1), BTree(5)), BTree(11, BTree(9)))

In [23]:
adjoin(t, 8)

BTree(8)

As we can see, `adjoin` returns a binary search tree that is containing only 8! What about the original tree?

It turns out that when checking whether the current label is greater or smaller than `v`, we need to return a `BTree` that includes the original tree. For example,

In [24]:
elif s.label < v:
    return BTree(s.label, s.left, adjoin(s.right, v))
elif s.label > v:
    return BTree(s.label, adjoin(s.left, v), s.right)

SyntaxError: invalid syntax (<ipython-input-24-bfb619a5b89e>, line 1)

Thus we have,

In [25]:
def adjoin(s, v): 
    if s is BTree.empty:
        return BTree(v) 
    elif s.label == v:
        return s
    elif s.label < v:
        return BTree(s.label, s.left, adjoin(s.right, v))
    elif s.label > v:
        return BTree(s.label, adjoin(s.left, v), s.right)

In [26]:
adjoin(t, 8) # Note that this doesn't mutate the original tree, `t`.

BTree(7, BTree(3, BTree(1), BTree(5)), BTree(11, BTree(9, BTree(8))))

In [27]:
print(adjoin(t, 8))

7
  3
    1
      None
      None
    5
      None
      None
  11
    9
      8
        None
        None
      None
    None


In [28]:
adjoin(t, 3)

BTree(7, BTree(3, BTree(1), BTree(5)), BTree(11, BTree(9)))

`adjoin`ing in this way doesn't guarantee a balanced structure. 

In [29]:
t = BTree.empty
for k in range(20):
    t = adjoin(t, k)

t

BTree(0, BTree.empty, BTree(1, BTree.empty, BTree(2, BTree.empty, BTree(3, BTree.empty, BTree(4, BTree.empty, BTree(5, BTree.empty, BTree(6, BTree.empty, BTree(7, BTree.empty, BTree(8, BTree.empty, BTree(9, BTree.empty, BTree(10, BTree.empty, BTree(11, BTree.empty, BTree(12, BTree.empty, BTree(13, BTree.empty, BTree(14, BTree.empty, BTree(15, BTree.empty, BTree(16, BTree.empty, BTree(17, BTree.empty, BTree(18, BTree.empty, BTree(19))))))))))))))))))))

In [30]:
print(t)

0
  None
  1
    None
    2
      None
      3
        None
        4
          None
          5
            None
            6
              None
              7
                None
                8
                  None
                  9
                    None
                    10
                      None
                      11
                        None
                        12
                          None
                          13
                            None
                            14
                              None
                              15
                                None
                                16
                                  None
                                  17
                                    None
                                    18
                                      None
                                      19
                                        None
                                        None


The problem was caused by the fact that we are trying to adjoin a sorted order. If we are trying to adjoin an arbitrary order instead, the result would be different. 

In [32]:
s = list(range(20))
s

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [33]:
from random import shuffle
shuffle(s)
s

[18, 4, 12, 19, 13, 17, 15, 10, 8, 1, 0, 6, 14, 3, 7, 5, 2, 11, 16, 9]

In [34]:
t = BTree.empty # Start with an empty tree
for k in s:
    t = adjoin(t, k)

In [35]:
t

BTree(18, BTree(4, BTree(1, BTree(0), BTree(3, BTree(2))), BTree(12, BTree(10, BTree(8, BTree(6, BTree(5), BTree(7)), BTree(9)), BTree(11)), BTree(13, BTree.empty, BTree(17, BTree(15, BTree(14), BTree(16)))))), BTree(19))

In [36]:
print(t)

18
  4
    1
      0
        None
        None
      3
        2
          None
          None
        None
    12
      10
        8
          6
            5
              None
              None
            7
              None
              None
          9
            None
            None
        11
          None
          None
      13
        None
        17
          15
            14
              None
              None
            16
              None
              None
          None
  19
    None
    None


As we can see, if we try to adjoin an arbitrary sequence of number, the result is not as deeply nested compared to if we adjoin an ordered sequence!

Searching through an element within this binary search tree would be faster compared to searching within a long linear tree. 

## Additional

An interesting topic in CS is how to keep binary search trees balanced. 